<a href="https://colab.research.google.com/github/aloiswirth/handson-ml2/blob/master/12b_Ragged_tensors_addon_to_chapter12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ragged tensors

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/ragged_tensor"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Setup


In [2]:
!pip install -q tf_nightly

     |████████████████████████████████| 322.9MB 43kB/s 
     |████████████████████████████████| 460kB 52.6MB/s 
     |████████████████████████████████| 3.0MB 44.9MB/s 


In [3]:
import math
import tensorflow as tf

## Overview

Your data comes in many shapes; your tensors should too.
*Ragged tensors* are the TensorFlow equivalent of nested variable-length
lists. They make it easy to store and process data with non-uniform shapes,
including:

*   Variable-length features, such as the set of actors in a movie.
*   Batches of variable-length sequential inputs, such as sentences or video
    clips.
*   Hierarchical inputs, such as text documents that are subdivided into
    sections, paragraphs, sentences, and words.
*   Individual fields in structured inputs, such as protocol buffers.

### What you can do with a ragged tensor
Ragged tensors are supported by more than a hundred TensorFlow operations, including math operations (such as tf.add and tf.reduce_mean), array operations (such as tf.concat and tf.tile), string manipulation ops (such as tf.substr), control flow operations (such as tf.while_loop and tf.map_fn), and many others:

In [4]:
digits = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])
words = tf.ragged.constant([["So", "long"], ["thanks", "for", "all", "the", "fish"]])

There are also a number of methods and operations that are specific to ragged tensors, including factory methods, conversion methods, and value-mapping operations. For a list of supported ops, see the **`tf.ragged` package documentation**.

Ragged tensors are supported by many TensorFlow APIs, including [Keras](https://www.tensorflow.org/guide/keras), [Datasets](https://www.tensorflow.org/guide/data), [tf.function](https://www.tensorflow.org/guide/function), [SavedModels](https://www.tensorflow.org/guide/saved_model), and [tf.Example](https://www.tensorflow.org/tutorials/load_data/tfrecord).  For more information, see the section on **TensorFlow APIs** below.

As with normal tensors, you can use Python-style indexing to access specific slices of a ragged tensor. For more information, see the section on **Indexing** below.

In [5]:
digits

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], []]>

In [6]:
print(digits[3])
digits[0]

tf.Tensor([6], shape=(1,), dtype=int32)


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([3, 1, 4, 1], dtype=int32)>

In [7]:
print(digits[0])

tf.Tensor([3 1 4 1], shape=(4,), dtype=int32)


In [8]:
print(digits[:, :2])

<tf.RaggedTensor [[3, 1], [], [5, 9], [6], []]>


In [9]:
print(digits[:, -2:])

<tf.RaggedTensor [[4, 1], [], [9, 2], [6], []]>


In [10]:
# Adding 3 to all numbers in the tensor:
print(digits + 3) 

<tf.RaggedTensor [[6, 4, 7, 4], [], [8, 12, 5], [9], []]>


In [11]:
print(tf.add(digits, 3))
# this adds 3 to all values

<tf.RaggedTensor [[6, 4, 7, 4], [], [8, 12, 5], [9], []]>


In [12]:
# Now calculate the average
print(tf.reduce_mean(digits, axis=1))

tf.Tensor([2.25              nan 5.33333333 6.                nan], shape=(5,), dtype=float64)


In [13]:
# Conatenation of two Ragged Tensors:
print(tf.concat([digits, [[5, 3]]], axis=0))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], [], [5, 3]]>


In [14]:
# Duplicate each component of a ragged tensor:
print(tf.tile(digits, [4, 1]))
print(tf.tile(digits, [1, 3]))
print(tf.tile(digits, [2, 3]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], [], [3, 1, 4, 1], [], [5, 9, 2], [6], [], [3, 1, 4, 1], [], [5, 9, 2], [6], [], [3, 1, 4, 1], [], [5, 9, 2], [6], []]>
<tf.RaggedTensor [[3, 1, 4, 1, 3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2, 5, 9, 2], [6, 6, 6], []]>
<tf.RaggedTensor [[3, 1, 4, 1, 3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2, 5, 9, 2], [6, 6, 6], [], [3, 1, 4, 1, 3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2, 5, 9, 2], [6, 6, 6], []]>


In [15]:
print(tf.map_fn(tf.math.square, digits))
# print(tf.map_fn(tf.math.exp(3.0) , digits))  ## doesn't work

<tf.RaggedTensor [[9, 1, 16, 1], [], [25, 81, 4], [36], []]>


In [16]:
words

<tf.RaggedTensor [[b'So', b'long'], [b'thanks', b'for', b'all', b'the', b'fish']]>

In [17]:
print(words[0])
words[1]

tf.Tensor([b'So' b'long'], shape=(2,), dtype=string)


<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'thanks', b'for', b'all', b'the', b'fish'], dtype=object)>

In [18]:
# substring operations on ragged string tensors
print(tf.strings.substr(words, 0, 2))
print(tf.strings.substr(words, 2, 4))

<tf.RaggedTensor [[b'So', b'lo'], [b'th', b'fo', b'al', b'th', b'fi']]>
<tf.RaggedTensor [[b'', b'ng'], [b'anks', b'r', b'l', b'e', b'sh']]>


Adding of ragged tensors (using +) can only be executed if the both tensors are of some type in all components. 

Excample [6] + [8, 9] does not work. See next example failing in the second line.

Error: Unable to broadcast: dimension size mismatch in dimension'
1
b'lengths='
4, 0, 3, 2, 0
b'dim_size='
4, 0, 3, 1, 0


In [19]:
print(digits + tf.ragged.constant([[1, 2, 3, 4], [], [5, 6, 7], [8], []]))

<tf.RaggedTensor [[4, 3, 7, 5], [], [10, 15, 9], [14], []]>


In [20]:
try:
  digits + tf.ragged.constant([[1, 2, 3, 4], [], [5, 6, 7], [8, 9], []])
except ValueError as exception:
  print(exception)
# except InvalidArgumentError as exception2:
#   print(exception2)
except:
  print("something else went wrong")

something else went wrong


If you need to perform an elementwise transformation to the values of a RaggedTensor, you can use tf.ragged.map_flat_values, which takes a function plus one or more arguments, and applies the function to transform the RaggedTensor's values.

In [21]:
times_two_plus_one = lambda x: x * 2 + 1
print(tf.ragged.map_flat_values(times_two_plus_one, digits))

<tf.RaggedTensor [[7, 3, 9, 3], [], [11, 19, 5], [13], []]>


In [22]:
exponent_three_plus_one = lambda x: x ** 3 + 1
print(tf.ragged.map_flat_values(exponent_three_plus_one, digits))

<tf.RaggedTensor [[28, 2, 65, 2], [], [126, 730, 9], [217], []]>


Conversion of ragged tensor to list of numpy nd array:

In [23]:
my_converted_list = digits.to_list()
print(my_converted_list)
for item in my_converted_list:
  print(item)

[[3, 1, 4, 1], [], [5, 9, 2], [6], []]
[3, 1, 4, 1]
[]
[5, 9, 2]
[6]
[]


In [24]:
np_converted_tensors = digits.numpy()
for my_array in np_converted_tensors:
  print(my_array)

[3 1 4 1]
[]
[5 9 2]
[6]
[]


### Constructing a ragged tensor
The simplest way to construct a ragged tensor is using `tf.ragged.constant`, which builds the `RaggedTensor` corresponding to a given nested Python `list` or numpy `array`:

In [25]:
sentences = tf.ragged.constant([
    ["Let's", "build", "some", "ragged", "tensors", "!"],
    ["We", "can", "use", "tf.ragged.constant", "."]])
print(sentences)
for i in sentences:
  print(i)

<tf.RaggedTensor [[b"Let's", b'build', b'some', b'ragged', b'tensors', b'!'], [b'We', b'can', b'use', b'tf.ragged.constant', b'.']]>
tf.Tensor([b"Let's" b'build' b'some' b'ragged' b'tensors' b'!'], shape=(6,), dtype=string)
tf.Tensor([b'We' b'can' b'use' b'tf.ragged.constant' b'.'], shape=(5,), dtype=string)


In [26]:
list_sentences = sentences.to_list()
list_sentences

[[b"Let's", b'build', b'some', b'ragged', b'tensors', b'!'],
 [b'We', b'can', b'use', b'tf.ragged.constant', b'.']]

In [27]:
paragraphs = tf.ragged.constant([
    [['I', 'have', 'a', 'cat'], ['His', 'name', 'is', 'Mat']],
    [['Do', 'you', 'want', 'to', 'come', 'visit'], ["I'm", 'free', 'tomorrow']],
])
print(paragraphs)

<tf.RaggedTensor [[[b'I', b'have', b'a', b'cat'], [b'His', b'name', b'is', b'Mat']], [[b'Do', b'you', b'want', b'to', b'come', b'visit'], [b"I'm", b'free', b'tomorrow']]]>


In [28]:
list_paragraphs = paragraphs.to_list()
list_paragraphs

[[[b'I', b'have', b'a', b'cat'], [b'His', b'name', b'is', b'Mat']],
 [[b'Do', b'you', b'want', b'to', b'come', b'visit'],
  [b"I'm", b'free', b'tomorrow']]]

#### Alternative Representations of Ragged Tensors
Ragged tensors can also be constructed by pairing flat *values* tensors with
*row-partitioning* tensors indicating how those values should be divided into
rows, using factory classmethods such as 
* `tf.RaggedTensor.from_value_rowids`,
* `tf.RaggedTensor.from_row_lengths`, and
* `tf.RaggedTensor.from_row_splits`.

##### 1.) `tf.RaggedTensor.from_value_rowids`
If you know which row each value belongs in, then you can build a `RaggedTensor` using a `value_rowids` row-partitioning tensor:

![value_rowids](https://www.tensorflow.org/images/ragged_tensors/value_rowids.png)

In [29]:
print(tf.RaggedTensor.from_value_rowids(
    values=[3, 1, 4, 1, 5, 9, 2],
    value_rowids=[0, 0, 0, 0, 2, 2, 3]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9], [2]]>


##### 2.) `tf.RaggedTensor.from_row_lengths`

If you know how long each row is, then you can use a `row_lengths` row-partitioning tensor:

![row_lengths](https://www.tensorflow.org/images/ragged_tensors/row_lengths.png)

In [30]:
print(tf.RaggedTensor.from_row_lengths(
    values=[3, 1, 4, 1, 5, 9, 2],
    row_lengths=[4, 0, 2, 1]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9], [2]]>


##### 3.) `tf.RaggedTensor.from_row_splits`

If you know the index where each row starts and ends, then you can use a `row_splits` row-partitioning tensor:

![row_splits](https://www.tensorflow.org/images/ragged_tensors/row_splits.png)

In [31]:
print(tf.RaggedTensor.from_row_splits(
    values=[3, 1, 4, 1, 5, 9, 2],
    row_splits=[0, 4, 4, 6, 7, 7]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9], [2], []]>


See the `tf.RaggedTensor` class documentation for a full list of factory methods.

Note: By default, these factory methods add assertions that the row partition tensor is well-formed and consistent with the number of values.  The `validate=False` parameter can be used to skip these checks if you can guarantee that the inputs are well-formed and consistent.

### What you can store in a ragged tensor

As with normal `Tensor`s, the values in a `RaggedTensor` must all have the same
type; and the values must all be at the same nesting depth (the *rank* of the
tensor):

In [32]:
print(tf.ragged.constant([["Hi"], ["How", "are", "you"]]))  # ok: type=string, rank=2

<tf.RaggedTensor [[b'Hi'], [b'How', b'are', b'you']]>


In [33]:
print(tf.ragged.constant([[[1, 2], [3]], [[4, 5]]]))        # ok: type=int32, rank=3

<tf.RaggedTensor [[[1, 2], [3]], [[4, 5]]]>


In [34]:
try:
  tf.ragged.constant([["one", "two"], [3, 4]])              # bad: multiple types
except ValueError as exception:
  print(exception)

Can't convert Python sequence with mixed types to Tensor.


In [35]:
try:
  tf.ragged.constant(["A", ["B", "C"]])                     # bad: multiple nesting depths
except ValueError as exception:
  print(exception)

all scalar values must have the same nesting depth


## Example use case

The following example demonstrates how `RaggedTensor`s can be used to construct
and combine unigram and bigram embeddings for a batch of variable-length
queries, using special markers for the beginning and end of each sentence.
For more details on the ops used in this example, see the `tf.ragged` package documentation.

In [36]:
queries = tf.ragged.constant([['Who', 'is', 'today', 'Dan', 'Smith'],
                              ['Pause'],
                              ['Will', 'it', 'rain', 'later', 'today']])
queries

<tf.RaggedTensor [[b'Who', b'is', b'today', b'Dan', b'Smith'], [b'Pause'], [b'Will', b'it', b'rain', b'later', b'today']]>

In [37]:
queries.shape

TensorShape([3, None])

In [38]:
# just to see how this works
len(queries.to_list())

3

In [39]:
# Create an embedding table.
num_buckets = 1024
embedding_size = 4
embedding_table = tf.Variable(
    tf.random.truncated_normal([num_buckets, embedding_size],
                       stddev=1.0 / math.sqrt(embedding_size)))

print(embedding_table)
'''
array([[ 0.7967023 ,  0.80937356, -0.933402  , -0.76543057],
       [-0.11630829,  0.14486092, -0.69844437, -0.6212817 ],
'''

<tf.Variable 'Variable:0' shape=(1024, 4) dtype=float32, numpy=
array([[-0.47374943, -0.9380833 , -0.9195592 ,  0.46555164],
       [ 0.11055991,  0.2858898 , -0.23720637,  0.01782364],
       [ 0.19101039, -0.12692028, -0.2644193 ,  0.37979347],
       ...,
       [-0.2769897 ,  0.6601249 , -0.2920366 , -0.63613415],
       [-0.3016212 , -0.45107308,  0.59042484, -0.6676856 ],
       [-0.5972573 ,  0.359846  ,  0.01489842, -0.05979504]],
      dtype=float32)>


'\narray([[ 0.7967023 ,  0.80937356, -0.933402  , -0.76543057],\n       [-0.11630829,  0.14486092, -0.69844437, -0.6212817 ],\n'

In [42]:
math.sqrt(embedding_size)

2.0

In [ ]:
for i in range(1024):
  for j in range(4):
    print(embedding_table[i , j])

In [44]:
# Look up the embedding for each word.
# word_buckets contains integers reqresenting the word strings
word_buckets = tf.strings.to_hash_bucket_fast(queries, num_buckets)
# word_embeddings contain a tensor representation of word bucket within the embedding table
word_embeddings = tf.nn.embedding_lookup(embedding_table, word_buckets)     # ①
print(word_buckets)
print(word_embeddings)

<tf.RaggedTensor [[633, 768, 547, 237, 309], [28], [847, 998, 635, 20, 547]]>
<tf.RaggedTensor [[[0.10146832466125488, -0.5626548528671265, -0.7980397939682007, 0.10221785306930542], [0.24285681545734406, 0.1617680937051773, 0.255990207195282, 0.14474603533744812], [-0.3706987798213959, -0.840350866317749, 0.3574962019920349, 0.06445838510990143], [-0.1280132234096527, -0.07512976974248886, 0.24972090125083923, 0.42566919326782227], [0.14002758264541626, 0.11549173295497894, -0.3013416826725006, -0.5129379630088806]], [[-0.16432859003543854, 0.047676727175712585, 0.48892122507095337, 0.38809067010879517]], [[0.3819596469402313, -0.5106041431427002, -0.5726322531700134, 0.2682870328426361], [0.29955390095710754, -0.21033115684986115, -0.17073477804660797, 0.19750995934009552], [0.34772780537605286, 0.12917667627334595, -0.38308560848236084, -0.6275220513343811], [0.7598094344139099, -0.14650900661945343, -0.8007323145866394, 0.09730249643325806], [-0.3706987798213959, -0.840350866317749

In [45]:
# Add markers to the beginning and end of each sentence.
marker = tf.fill([queries.nrows(), 1], '#')
marker

<tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b'#'],
       [b'#'],
       [b'#']], dtype=object)>

In [46]:
padded = tf.concat([marker, queries, marker], axis=1)                       # ②
print(padded)

<tf.RaggedTensor [[b'#', b'Who', b'is', b'today', b'Dan', b'Smith', b'#'], [b'#', b'Pause', b'#'], [b'#', b'Will', b'it', b'rain', b'later', b'today', b'#']]>


In [47]:
# Build word bigrams & look up embeddings.
bigrams = tf.strings.join([padded[:, :-1], padded[:, 1:]], separator='+')   # ③
print(bigrams)

<tf.RaggedTensor [[b'#+Who', b'Who+is', b'is+today', b'today+Dan', b'Dan+Smith', b'Smith+#'], [b'#+Pause', b'Pause+#'], [b'#+Will', b'Will+it', b'it+rain', b'rain+later', b'later+today', b'today+#']]>


In [48]:
bigram_buckets = tf.strings.to_hash_bucket_fast(bigrams, num_buckets)
print(bigram_buckets)


<tf.RaggedTensor [[939, 196, 460, 372, 281, 176], [597, 723], [853, 293, 853, 704, 749, 372]]>


In [51]:
bigram_embeddings = tf.nn.embedding_lookup(embedding_table, bigram_buckets) # ④
print(bigram_embeddings)
print(bigram_embeddings.shape)

<tf.RaggedTensor [[[0.5531015992164612, 0.06233026832342148, -0.38662803173065186, -0.5691091418266296], [-0.8586159944534302, -0.5492018461227417, -0.06206214427947998, -0.1813337504863739], [-0.061614081263542175, -0.20091888308525085, 0.41288983821868896, -0.6746004819869995], [0.3288642168045044, -0.12639276683330536, -0.32625681161880493, -0.02704225666821003], [-0.9494671821594238, -0.1673433482646942, 0.12439165264368057, 0.7177324891090393], [-0.08185794204473495, -0.6300257444381714, -0.23338942229747772, 0.3234575390815735]], [[-0.5965119004249573, 0.9658960700035095, 0.5366710424423218, 0.7602482438087463], [-0.2830057740211487, -0.13205184042453766, -0.40244248509407043, 0.08063250780105591]], [[0.2646404206752777, -0.10149259865283966, 0.16721482574939728, -0.5372437834739685], [-0.13156656920909882, 0.0776573121547699, 0.07330163568258286, -0.05435384437441826], [0.2646404206752777, -0.10149259865283966, 0.16721482574939728, -0.5372437834739685], [0.19541554152965546, -0.

The above print shows 3, None, 4. 3 is the number of sentences, 'None' is indicating that the number of words per sentance varies, and thus the dimension is ragged. 4 indicates the embedding size. 

In [53]:
# Find the average embedding for each sentence
all_embeddings = tf.concat([word_embeddings, bigram_embeddings], axis=1)    # ⑤
print(all_embeddings.shape)
all_embeddings

(3, None, 4)


<tf.RaggedTensor [[[0.10146832466125488, -0.5626548528671265, -0.7980397939682007, 0.10221785306930542], [0.24285681545734406, 0.1617680937051773, 0.255990207195282, 0.14474603533744812], [-0.3706987798213959, -0.840350866317749, 0.3574962019920349, 0.06445838510990143], [-0.1280132234096527, -0.07512976974248886, 0.24972090125083923, 0.42566919326782227], [0.14002758264541626, 0.11549173295497894, -0.3013416826725006, -0.5129379630088806], [0.5531015992164612, 0.06233026832342148, -0.38662803173065186, -0.5691091418266296], [-0.8586159944534302, -0.5492018461227417, -0.06206214427947998, -0.1813337504863739], [-0.061614081263542175, -0.20091888308525085, 0.41288983821868896, -0.6746004819869995], [0.3288642168045044, -0.12639276683330536, -0.32625681161880493, -0.02704225666821003], [-0.9494671821594238, -0.1673433482646942, 0.12439165264368057, 0.7177324891090393], [-0.08185794204473495, -0.6300257444381714, -0.23338942229747772, 0.3234575390815735]], [[-0.16432859003543854, 0.047676

In [54]:
avg_embedding = tf.reduce_mean(all_embeddings, axis=1)                      # ⑥
print(avg_embedding)

tf.Tensor(
[[-0.09854078 -0.2556753  -0.06429356 -0.01697655]
 [-0.34794876  0.29384032  0.20771663  0.40965715]
 [ 0.20087905 -0.18121055 -0.2464324  -0.2008891 ]], shape=(3, 4), dtype=float32)


![ragged_example](https://www.tensorflow.org/images/ragged_tensors/ragged_example.png)

## Ragged and uniform dimensions

A ***ragged dimension*** is a dimension whose slices may have different lengths. For example, the
inner (column) dimension of `rt=[[3, 1, 4, 1], [], [5, 9, 2], [6], []]` is
ragged, since the column slices (`rt[0, :]`, ..., `rt[4, :]`) have different
lengths. Dimensions whose slices all have the same length are called *uniform
dimensions*.

The outermost dimension of a ragged tensor is always uniform, since it consists
of a single slice (and so there is no possibility for differing slice
lengths).  The remaining dimensions may be either ragged or uniform. For
example, we might store the word embeddings for
each word in a batch of sentences using a ragged tensor with shape
`[num_sentences, (num_words), embedding_size]`, where the parentheses around
`(num_words)` indicate that the dimension is ragged.

![sent_word_embed](https://www.tensorflow.org/images/ragged_tensors/sent_word_embed.png)

Ragged tensors may have multiple ragged dimensions. For example, we could store
a batch of structured text documents using a tensor with shape `[num_documents,
(num_paragraphs), (num_sentences), (num_words)]` (where again parentheses are
used to indicate ragged dimensions).

As with `tf.Tensor`, the ***rank*** of a ragged tensor is its total number of dimensions (including both ragged and uniform dimensions).
A ***potentially ragged tensor*** is a value that might be
either a `tf.Tensor` or a `tf.RaggedTensor`.

When describing the shape of a RaggedTensor, ragged dimensions are conventionally indicated by
enclosing them in parentheses. For example, as we saw above, the shape of a 3-D
RaggedTensor that stores word embeddings for each word in a batch of sentences
can be written as `[num_sentences, (num_words), embedding_size]`.

The `RaggedTensor.shape` attribute returns a `tf.TensorShape` for a
ragged tensor, where ragged dimensions have size `None`:

In [55]:
tf.ragged.constant([["Hi"], ["How", "are", "you"]]).shape

TensorShape([2, None])

The method `tf.RaggedTensor.bounding_shape` can be used to find a tight
bounding shape for a given `RaggedTensor`:

In [56]:
print(tf.ragged.constant([["Hi"], ["How", "are", "you"]]).bounding_shape())

tf.Tensor([2 3], shape=(2,), dtype=int64)


## Ragged vs. sparse

A ragged tensor should *not* be thought of as a type of sparse tensor.  In particular, sparse tensors are *efficient encodings for tf.Tensor*, that model the same data in a compact format; but ragged tensor is an *extension to tf.Tensor*, that models an expanded class of data.  This difference is crucial when defining operations:

* Applying an op to a sparse or dense tensor should always give the same result.
* Applying an op to a ragged or sparse tensor may give different results.

As an illustrative example, consider how array operations such as `concat`,
`stack`, and `tile` are defined for ragged vs. sparse tensors. Concatenating
ragged tensors joins each row to form a single row with the combined length:

![ragged_concat](https://www.tensorflow.org/images/ragged_tensors/ragged_concat.png)


In [57]:
ragged_x = tf.ragged.constant([["John"], ["a", "big", "dog"], ["my", "cat"]])
ragged_y = tf.ragged.constant([["fell", "asleep"], ["barked"], ["is", "fuzzy"]])
print(tf.concat([ragged_x, ragged_y], axis=1))

<tf.RaggedTensor [[b'John', b'fell', b'asleep'], [b'a', b'big', b'dog', b'barked'], [b'my', b'cat', b'is', b'fuzzy']]>


But concatenating sparse tensors is equivalent to concatenating the corresponding dense tensors,
as illustrated by the following example (where Ø indicates missing values):

![sparse_concat](https://www.tensorflow.org/images/ragged_tensors/sparse_concat.png)

In [60]:
sparse_x = ragged_x.to_sparse()
print(sparse_x)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 0]
 [1 1]
 [1 2]
 [2 0]
 [2 1]], shape=(6, 2), dtype=int64), values=tf.Tensor([b'John' b'a' b'big' b'dog' b'my' b'cat'], shape=(6,), dtype=string), dense_shape=tf.Tensor([3 3], shape=(2,), dtype=int64))


In [63]:
sparse_y = ragged_y.to_sparse()
print(sparse_y)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [1 0]
 [2 0]
 [2 1]], shape=(5, 2), dtype=int64), values=tf.Tensor([b'fell' b'asleep' b'barked' b'is' b'fuzzy'], shape=(5,), dtype=string), dense_shape=tf.Tensor([3 2], shape=(2,), dtype=int64))


In [64]:
sparse_result = tf.sparse.concat(sp_inputs=[sparse_x, sparse_y], axis=1)
print(tf.sparse.to_dense(sparse_result, ''))

tf.Tensor(
[[b'John' b'' b'' b'fell' b'asleep']
 [b'a' b'big' b'dog' b'barked' b'']
 [b'my' b'cat' b'' b'is' b'fuzzy']], shape=(3, 5), dtype=string)


For another example of why this distinction is important, consider the
definition of “the mean value of each row” for an op such as `tf.reduce_mean`.
For a ragged tensor, the mean value for a row is the sum of the
row’s values divided by the row’s width.
But for a sparse tensor, the mean value for a row is the sum of the
row’s values divided by the sparse tensor’s overall width (which is
greater than or equal to the width of the longest row).